In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF, StopWordsRemover,RegexTokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, IndexToString, StandardScaler
import shutil
import os
from pyspark.ml.stat import Correlation
import pandas as pd
import seaborn as sns

In [2]:
#spark.stop()

In [3]:
#open Spark Session
spark = SparkSession.builder.appName('prices_houses_v5').master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
#spark.sparkContext.addPyFile("sparkxgb.zip")

In [4]:
#read source I remove last 10 entries for predictions 
data=spark.read.csv("train.csv", inferSchema=True,sep=',',header=True)
datatest=spark.read.csv("test.csv", schema=data.schema,sep=',',header=True)

In [5]:
len(data.columns),len(datatest.columns)

(81, 81)

In [6]:
columnListS = [item[0] for item in data.dtypes if item[1].startswith('string')]
columnListI = [item[0] for item in data.dtypes if item[1].startswith('int')]
columnListS_T = [item[0] for item in datatest.dtypes if item[1].startswith('string')]
columnListI_T = [item[0] for item in datatest.dtypes if item[1].startswith('int')]

In [7]:
datatest=datatest.fillna("NO_Value",subset=columnListS_T)
#datatest=datatest.fillna(0,subset=columnListI_T)
data=data.fillna("NO_Value",subset=columnListS)

In [8]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",handleInvalid="keep").fit(data) for column in list(columnListS) ]
pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(data).transform(data)

indexers2 = [StringIndexer(inputCol=column, outputCol=column+"_index",handleInvalid="keep").fit(datatest) for column in list(columnListS_T) ]
pipeline2 = Pipeline(stages=indexers2)
df_rt = pipeline2.fit(datatest).transform(datatest)

In [9]:
columnListS_Cat=[item for item in df_r.columns if str(item).endswith('_index')]

In [10]:
len(columnListS_Cat)+len(columnListI)

81

In [11]:

lr_data = df_r.select(sf.col("SalePrice").alias("label"), *columnListI,*columnListS_Cat).dropna()

columns = lr_data.columns

In [12]:
lr_datat = df_rt.select(sf.col("SalePrice").alias("label"), *columnListI_T,*columnListS_Cat)
columnst = lr_datat.columns

In [13]:
lr_datat=lr_datat.fillna(0)

In [14]:
lr_datat.count()

1459

In [15]:
resumen2=lr_data.toPandas()
resumen3=lr_datat.toPandas()
resumen2.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
label,1460.0,180921.195890,79442.502883,34900.0,129975.00,163000.0,214000.00,755000.0
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0
BsmtFinSF2,1460.0,46.549315,161.319273,0.0,0.00,0.0,0.00,1474.0


In [16]:
resumen3.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
label,1459.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
Id,1459.0,2185.279644,432.726126,0.0,1822.5,2188.0,2553.5,2919.0
MSSubClass,1459.0,57.302947,42.802319,0.0,20.0,50.0,70.0,190.0
LotArea,1459.0,9776.661412,4877.960604,0.0,7379.5,9392.0,11507.5,56600.0
OverallQual,1459.0,6.069225,1.461440,0.0,5.0,6.0,7.0,10.0
OverallCond,1459.0,5.540096,1.140477,0.0,5.0,5.0,6.0,9.0
YearBuilt,1459.0,1967.363263,94.349573,0.0,1953.0,1973.0,2001.0,2010.0
YearRemodAdd,1459.0,1979.588074,92.332975,0.0,1963.0,1992.0,2004.0,2010.0
BsmtFinSF1,1459.0,438.527073,455.393086,0.0,0.0,350.0,752.0,4010.0
BsmtFinSF2,1459.0,52.583276,176.698671,0.0,0.0,0.0,0.0,1526.0


In [17]:
corr=resumen2.corr()

In [18]:
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

In [19]:
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [20]:
resumen2.columns

Index(['label', 'Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice',
       'MSZoning_index', 'LotFrontage_index', 'Street_index', 'Alley_index',
       'LotShape_index', 'LandContour_index', 'Utilities_index',
       'LotConfig_index', 'LandSlope_index', 'Neighborhood_index',
       'Condition1_index', 'Condition2_index', 'BldgType_index',
       'HouseStyle_index', 'RoofStyle_index', 'RoofMatl_index',
       'Exterior1st_index', 'Exterior2nd_index', 'MasVnrType_index',
       'MasVnrArea_index', 'ExterQual_index', 'ExterCond_ind

In [21]:
features_extra=['OverallQual','YearBuilt','YearRemodAdd','TotalBsmtSF','1stFlrSF','GrLivArea','FullBath','TotRmsAbvGrd','GarageCars','GarageArea','ExterQual_index']

In [55]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from scipy.stats import skew
from collections import OrderedDict
import numpy as np

In [27]:
df_train=resumen2[features_extra]
df_test=resumen3[features_extra]

In [28]:
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df_train)
scaled_df = pd.DataFrame(scaled_df, columns=features_extra)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(scaled_df, resumen2['label'], test_size=0.20, random_state=42)

In [34]:
import xgboost
from xgboost import plot_importance

In [35]:
model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

In [36]:
model.fit(X_train,y_train)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0, learning_rate=0.07, max_delta_step=0,
       max_depth=3, min_child_weight=1.5, missing=None, n_estimators=10000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
       silent=True, subsample=0.6)

In [43]:
#for tuning parameters
parameters_for_testing = {
    'colsample_bytree':[0.4,0.6,0.8],
    'gamma':[0,0.03,0.1,0.3],
    'min_child_weight':[1.5,6,10],
    'learning_rate':[0.1,0.07],
    'max_depth':[3,5],
    'n_estimators':[10000],
    'reg_alpha':[1e-5, 1e-2,  0.75],
    'reg_lambda':[1e-5, 1e-2, 0.45],
    'subsample':[0.6,0.95]  
}

                   
xgb_model = xgboost.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,
     min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)

gsearch1 = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')
gsearch1.fit(X_train,y_train)
print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

Fitting 3 folds for each of 2592 candidates, totalling 7776 fits


[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   22.0s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   46.7s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  2.6min
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  3.4min
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:  4.4min
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:  5.6min
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:  6.8min
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:  8.0min
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:  9.3min
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed: 10.8min
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed: 12.3min
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed: 13.8min
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed: 16.2min
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed: 19.1min
[Parallel(n_jobs=6)]: Do

[mean: -1126795250.68597, std: 151299208.16993, params: {'colsample_bytree': 0.4, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1.5, 'n_estimators': 10000, 'reg_alpha': 1e-05, 'reg_lambda': 1e-05, 'subsample': 0.6}, mean: -1060308655.35342, std: 143810189.49095, params: {'colsample_bytree': 0.4, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1.5, 'n_estimators': 10000, 'reg_alpha': 1e-05, 'reg_lambda': 1e-05, 'subsample': 0.95}, mean: -1128619788.44726, std: 157247181.35718, params: {'colsample_bytree': 0.4, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1.5, 'n_estimators': 10000, 'reg_alpha': 1e-05, 'reg_lambda': 0.01, 'subsample': 0.6}, mean: -1048836201.57415, std: 138144045.03041, params: {'colsample_bytree': 0.4, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1.5, 'n_estimators': 10000, 'reg_alpha': 1e-05, 'reg_lambda': 0.01, 'subsample': 0.95}, mean: -1133690934.67244, std: 163572907.175

In [44]:
best_xgb_model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0.03,                 
                 learning_rate=0.01,
                 max_depth=5,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=1e-05,
                 reg_lambda=1e-05,
                 subsample=0.95)
#                 seed=42)
best_xgb_model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0.03, learning_rate=0.01,
       max_delta_step=0, max_depth=5, min_child_weight=1.5, missing=None,
       n_estimators=10000, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=1e-05, reg_lambda=1e-05,
       scale_pos_weight=1, seed=None, silent=True, subsample=0.95)

In [50]:
ypred = best_xgb_model.predict(X_test)

In [58]:
RMSE= np.sqrt(mean_squared_error(y_test, ypred))

In [56]:
r2_score(y_test, ypred)

0.88476591236752655

In [59]:
print(RMSE)

29730.1705735


In [60]:
resumen3['Prediction'] = best_xgb_model.predict(df_test)

In [61]:
filename = 'submission.csv'
pd.DataFrame({'Id': resumen3.Id, 'SalePrice': resumen3.Prediction}).to_csv(filename, index=False)

In [63]:
print(resumen3['Prediction'].head())
print(resumen3['Prediction'].count())

0    248527.078125
1    248527.078125
2    248527.078125
3    248527.078125
4    239355.234375
Name: Prediction, dtype: float32
1459
